In [1072]:
library(ape)
library(seqinr)
library(phangorn)

### Remove genotypes with N<50 and add in the reference sequences

In [1073]:
# reference sequence for each genotype
# A: FJ692557
# B: GU815637
# C: GQ377617
# D: NC_003977
# E: GQ161817
# F: HM585194

In [1078]:
seqs = read.dna('~/hbv_covar3/QC/step3_phyloCheck/HBV_ALLgeno_nodup05_mafft_phyloFilter.fasta', format = 'fasta', as.character = T)
msa_genotypes = unlist(lapply(strsplit(rownames(seqs), '_'), function(x) x[2]))
analyse_genotype = c('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H')
                              
# first Let's remove genotypes under 50 (G and H)
table(msa_genotypes)
rm_genotypes = c('G', 'H')
analyse_genotype = analyse_genotype[-which(analyse_genotype %in% rm_genotypes)]
seqs = seqs[-which(msa_genotypes %in% rm_genotypes),]

##########################
# set up for realignment #
##########################
                              
# remove the gaps
seqs = apply(seqs, 1, function(x){
    x = x[-which(x == '-')]
    return(x)
})
                              
# then add the reference sequence into the mix
ref_id = list(A = 'FJ692557', B = 'GU815637', C = 'GQ377617', D = 'NC_003977', E = 'GQ161817', F = 'HM585194' )
for (genotype in analyse_genotype){
    if (paste0(ref_id[[genotype]], '_', genotype) %in% rownames(seqs)){
        message(paste0('PROBLEM: reference sequence', ref_id[[genotype]] ,'already in the msa!!'))
        break
    }else{
        ref_file = paste0('~/hbv_covar3/raw_data/', ref_id[[genotype]], '.fasta')
        ref_seq = read.dna(ref_file, as.character = T, format = 'fasta')
        seqs[[paste0('ref_', ref_id[[genotype]], '_', genotype)]] = ref_seq[1,]
    }
}
write.dna(seqs, '~/hbv_covar3/QC/step4_translate_gene/HBV_ALLgeno_phyloFilter_ref.fasta', format = 'fasta', colsep = '')

msa_genotypes
   A    B    C    D    E    F    G    H 
 453 1072 1674  841  261  136    7   18 

Sequences were then align using mafft <br>
/apps/well/mafft/7.149/bin/mafft HBV_ALLgeno_phyloFilter_ref.fasta > HBV_ALLgeno_phyloFilter_ref_mafft.fasta

In [1079]:
gene_ref_pos = list()
gene_ref_pos[['A']] = list(
    S = c(2854:3221,1:835),
    X = c(1374:1838),
    C = c(1814:2458),
    P = c(2307:3221,1:1623)
)
gene_ref_pos[['B']] = list(
    S = c(2848:3215,1:835),
    X = c(1374:1838),
    C = c(1814:2452),
    P = c(2307:3215,1:1623)
)
gene_ref_pos[['C']] = list(
    S = c(2848:3215,1:835),
    X = c(1374:1838),
    C = c(1814:2452),
    P = c(2307:3215,1:1623)
)
gene_ref_pos[['D']] = list(
    S = c(2850:3182,1:837),
    X = c(1376:1840),
    C = c(1816:2454),
    P = c(2309:3182,1:1625)
)
gene_ref_pos[['E']] = list(
    S = c(2848:3212,1:835),
    X = c(1374:1838),
    C = c(1814:2452),
    P = c(2307:3212,1:1623)
)
gene_ref_pos[['F']] = list(
    S = c(2848:3215,1:835),
    X = c(1374:1838),
    C = c(1814:2452),
    P = c(2307:3215,1:1623)
)

In [1080]:
seqs = read.dna('~/hbv_covar3/QC/step4_translate_gene/HBV_ALLgeno_phyloFilter_ref_mafft.fasta', format = 'fasta', as.character = T)

In [1125]:
ref_msa_id = vector()
for(genotype in analyse_genotype){
    ref_msa_id[genotype] = paste0('ref_', ref_id[[genotype]], '_', genotype)
}

# adjust gene pos to msa
ref_msa_originPos = list()
for(genotype in analyse_genotype){
    ref_msa_seq = seqs[ref_msa_id[genotype],]
    ref_msa_originPos[[genotype]] = rep(NA, length(ref_msa_seq))
    ind = 1
    for(i in 1:length(ref_msa_seq)){
        nt = ref_msa_seq[i]
        if (nt != '-'){
            ref_msa_originPos[[genotype]][i] = ind
            ind = ind + 1
        }
    }
}


In [1082]:
# get the msa pos
gene_msa_pos = list()
for (genotype in analyse_genotype){
    gene_msa_pos[[genotype]] = list()
    for (gene in names(gene_ref_pos[['A']])){
        ref_pos = gene_ref_pos[[genotype]][[gene]]
        start = ref_pos[1]
        end = ref_pos[length(ref_pos)]
        if (end > start){
            msa_start = which(ref_msa_originPos[[genotype]] == start)
            msa_end = which(ref_msa_originPos[[genotype]] == end)
            gene_msa_pos[[genotype]][[gene]] = msa_start:msa_end
        }else {
            msa_start = which(ref_msa_originPos[[genotype]] == start)
            msa_end = which(ref_msa_originPos[[genotype]] == end)
            gene_msa_pos[[genotype]][[gene]] = c(msa_start:ncol(seqs), 1:msa_end)
        }
    }
}

In [1124]:
saveRDS(gene_msa_pos, '~/hbv_covar3/QC/step4_translate_gene/gene_msa_pos.rds')

In [1086]:
gene = 'X'
sum(gene_msa_pos[['A']][[gene]] != gene_msa_pos[['B']][[gene]])
sum(gene_msa_pos[['B']][[gene]] != gene_msa_pos[['C']][[gene]])
sum(gene_msa_pos[['C']][[gene]] != gene_msa_pos[['D']][[gene]])
sum(gene_msa_pos[['D']][[gene]] != gene_msa_pos[['E']][[gene]])
sum(gene_msa_pos[['E']][[gene]] != gene_msa_pos[['F']][[gene]])

# For each gene, the msa position is the same across all genotypes, which is good!

[1] 0

[1] 0

[1] 0

[1] 0

[1] 0

In [1088]:
# extract the sequence and reomve gap to translate
gene_AAseqs = list()
gene_NTseqs = list()
for (gene in names(gene_ref_pos[['A']])){
    gene_pos = gene_msa_pos[['A']][[gene]] # all genotype map to the same position, does matter which one we choose here
    gene_msa_seq = seqs[,gene_pos]
    write.FASTA(as.DNAbin(gene_msa_seq),
                paste0('~/hbv_covar3/QC/step4_translate_gene/HBV_gene', gene, '_mafft_seqs.fasta'))
    gene_NTseqs[[gene]] = apply(gene_msa_seq, 1, function(x) x[-which(x == '-')])
    gene_AAseqs[[gene]] = lapply(gene_NTseqs[[gene]], translate)
    write.FASTA(as.DNAbin(gene_NTseqs[[gene]]),
               paste0('~/hbv_covar3/QC/step4_translate_gene/HBV_gene', gene, '_seqs.fasta'))
    write.FASTA(as.AAbin(gene_AAseqs[[gene]]), 
                paste0('~/hbv_covar3/QC/step4_translate_gene/HBV_gene', gene, '_AAseqs.fasta'))
}

Align the protein sequence for each gene using MAFFt, replace stop codon with ambiguous codon B for MAFFT to align it as well.<br>
example:<br>
sed 's/\*/B/g' HBV_geneC_AAseqs.fasta > temp.fasta<br>
/apps/well/mafft/7.149/bin/mafft temp.fasta > HBV_geneC_AAseqs_mafft.fasta

In [1097]:
#####################
# some sanity check #
#####################

# compare with HBVDB sequences
gene = 'X'
hbvDB_fasta = ''
if (gene == 'S'){
    hbvDB_fasta = '~/hbv_covar3/raw_data/PreS1_hbvDB.fas'
} else if (gene == 'C'){
    hbvDB_fasta = '~/hbv_covar3/raw_data/PreC_hbvDB.fas'
} else if (gene == 'X'){
    hbvDB_fasta = '~/hbv_covar3/raw_data/X_hbvDB.fas'
} else if (gene == 'P'){
    hbvDB_fasta = '~/hbv_covar3/raw_data/P_hbvDB.fas'
}

hbvDB = read.dna(hbvDB_fasta, format = 'fasta', as.character = T)
hbvDB.names = unlist(lapply(strsplit(names(hbvDB), '\\|'), function(x) x[3]))
hbvDB.names = unlist(lapply(strsplit(hbvDB.names, '_'), function(x) x[1]))

# does it starts with starting codon?
message('start codon check...')
start_codon_check = unlist(lapply(gene_AAseqs[[gene]], function(x) ifelse(x[1]=='M', 1, 0)))
table(start_codon_check)
start_codon_ind = names(gene_AAseqs[['S']][which(start_codon_check == 0)])
start_codon_ind = gsub('_.', '', start_codon_ind)
check_names = start_codon_ind[which(start_codon_ind %in% hbvDB.names)]
hbvDB_check = hbvDB[which(hbvDB.names %in% check_names)]
for(i in 1:length(hbvDB_check)){
    first_codon = paste(hbvDB_check[[i]][1:3], collapse = '')
    if (first_codon == 'atg'){
        print(first_codon)
    }
}
             
# does it end with stop codon?
message('stop codon check...')
stop_codon_check = unlist(lapply(gene_AAseqs[[gene]], function(x) ifelse(x[length(x)]=='*', 1, 0)))
table(stop_codon_check)
stop_codon_ind = names(gene_AAseqs[['S']][which(stop_codon_check == 0)])
stop_codon_ind = gsub('_.', '', stop_codon_ind)
check_names = stop_codon_ind[which(stop_codon_ind %in% hbvDB.names)]
hbvDB_check = hbvDB[which(hbvDB.names %in% check_names)]
for(i in 1:length(hbvDB_check)){
    last_codon = tail(translate(hbvDB_check[[i]]), n = 1)
    if (last_codon == '*'){
        print(names(hbvDB_check)[i])
        print(last_codon)
    }
}

# is there stop codon in the middle?
hbvDB = read.dna(hbvDB_fasta, format = 'fasta', as.character = T)
hbvDB.names = unlist(lapply(strsplit(names(hbvDB), '\\|'), function(x) x[3]))
hbvDB.names = unlist(lapply(strsplit(hbvDB.names, '_'), function(x) x[1]))

message('middel stop codon check...')
stop_codon_check = unlist(lapply(gene_AAseqs[[gene]], function(x) ifelse('*' %in% x[-length(x)], 0, 1)))
table(stop_codon_check)
stop_codon_ind = names(gene_AAseqs[['S']][which(stop_codon_check == 0)])
stop_codon_ind = gsub('_.', '', stop_codon_ind)
check_names = stop_codon_ind[which(stop_codon_ind %in% hbvDB.names)]
hbvDB_check = hbvDB[which(hbvDB.names %in% check_names)]
for(i in 1:length(hbvDB_check)){
    AA_seq = translate(hbvDB_check[[i]])
    if (!('*' %in% AA_seq[-length(AA_seq)])){
        print(names(hbvDB_check)[i])
    }
}


start codon check...



start_codon_check
   0    1 
   7 4436 

stop codon check...



stop_codon_check
   0    1 
  15 4428 

middel stop codon check...



stop_codon_check
   0    1 
  52 4391 

In [1129]:
test1 = c(1,NA,2, NA, 3)
test2 = c(1,2, 2, NA, 1)
length(which(test1 != test2))

[1] 1

In [1098]:
##################################
# detect frame-shifted sequences #
##################################
get_codon_pos = function(seq){
    result = rep(NA, length(seq))
    codon_pos = 0
    for(i in 1:length(seq)){
        nt = seq[i]
        if (nt != '-'){
            codon_pos = codon_pos + 1
            if(codon_pos == 4){
                codon_pos = 1
            }
            result[i] = codon_pos
        }
    }
    return(result)
}

frame_shift_lst = list()
for (gene in c('S', 'P', 'C', 'X')){
    frame_shift_lst[[gene]] = vector()
    # get codon position for each reference
    gene_pos = gene_msa_pos[['A']][[gene]]
    ref_codon_pos = lapply(analyse_genotype, function(x) {
        ref_seq = seqs[ref_msa_id[x],gene_pos]
        get_codon_pos(ref_seq)
    })
    names(ref_codon_pos) = analyse_genotype

    # get consensus amino acid sequence for each genotype
    aa_seqs_align_file = aa_seqs_align_file = paste0('~/hbv_covar3/QC/step4_translate_gene/HBV_gene', gene, '_AAseqs_mafft.fasta')
    aa_seqs_align = do.call(rbind, read.fasta(aa_seqs_align_file, seqtype = 'AA'))
    msa_genotypes = unlist(lapply(strsplit(rownames(seqs), '_'), function(x) x[2]))
    consensus_seq = lapply(analyse_genotype, function(x){
        cur_seqs = aa_seqs_align[which(msa_genotypes == x),]
        return(consensus(cur_seqs))
    })
    names(consensus_seq) = analyse_genotype
                                  

    for(i in 1:nrow(seqs)){
        name = rownames(seqs)[i]
        seq = seqs[i,gene_pos]
        aa_seq_align = aa_seqs_align[name,]
        genotype = strsplit(name, '_')[[1]][2]
        seq_codon_pos = get_codon_pos(seq)
        diff_codon = which(seq_codon_pos != ref_codon_pos[[genotype]])
        diff_consensus = which(aa_seq_align != consensus_seq[[genotype]])
        # if frameshift on 20% of the alignment, regard as frameshift
        threshold_codon = round(length(seq)/5)
        # if mismatch on 20% of the sequence alignment, regard as frameshift
        threshold_aa = round(length(aa_seq_align)/5)
        if (length(diff_codon) > threshold_codon | length(diff_consensus) > threshold_aa){
            frame_shift_lst[[gene]] = c(frame_shift_lst[[gene]], i)
        }# else{
        #     #stop codon appear in the middle of sequence, and is not a "special case", regard as frame shift
        #     #allow stop codon to be near the 10% end of the alignment
        #    end_length = round(length(aa_seq_align) * 0.9)
        #    aa_seq_strip = aa_seq_align[-(end_length:length(aa_seq_align))]
        #    stop_codon_ind = which(aa_seq_strip == 'B')
        #    if ('B' %in% aa_seq_strip){
        #        if (gene == 'C' & all(stop_codon_ind < 31)){
        #            # special case: C gene, and the stop codon appear in preCore
        #            next
        #        } else if (gene == 'S' & all(stop_codon_ind < 272)){
        #            # special case: S gene, and the stop codon appear in preS1/preS2
        #            next
        #        } else{
        #            print(paste0(gene, ': ', stop_codon_ind))
        #            frame_shift_lst[[gene]] = c(frame_shift_lst[[gene]], i)
        #        }
        #    }
        #}
    }
}


In [1122]:
# write the sequences after removing the frame-shifted sequences
for(gene in c('S', 'P', 'C', 'X')){
    frame_shift_name = rownames(seqs)[frame_shift_lst[[gene]]]
    if (length(frame_shift_name) != 0){
        gene_aa_align = read.FASTA(paste0('~/hbv_covar3/QC/step4_translate_gene/HBV_gene', gene, '_AAseqs_mafft.fasta'), type = 'AA')
        gene_aa_align_noShift = gene_aa_align[-which(names(gene_aa_align) %in% frame_shift_name)]
        write.FASTA(gene_aa_align_noShift, paste0('~/hbv_covar3/QC/step4_translate_gene/HBV_gene', gene, '_AAseqs_mafft_noShift.fasta'))

        gene_aa_seqs = read.FASTA(paste0('~/hbv_covar3/QC/step4_translate_gene/HBV_gene', gene, '_AAseqs.fasta'), type = 'AA')
        gene_aa_seqs_noShift = gene_aa_seqs[-which(names(gene_aa_seqs) %in% frame_shift_name)]
        write.FASTA(gene_aa_seqs_noShift, paste0('~/hbv_covar3/QC/step4_translate_gene/HBV_gene', gene, '_AAseqs_noShift.fasta'))

        gene_nt_seqs = read.FASTA(paste0('~/hbv_covar3/QC/step4_translate_gene/HBV_gene', gene, '_seqs.fasta'), type = 'AA')
        gene_nt_seqs_noShift = gene_nt_seqs[-which(names(gene_nt_seqs) %in% frame_shift_name)]
        write.FASTA(gene_nt_seqs_noShift, paste0('~/hbv_covar3/QC/step4_translate_gene/HBV_gene', gene, '_seqs_noShift.fasta'))
    }else{
        system(paste0('cp ', '~/hbv_covar3/QC/step4_translate_gene/HBV_gene', gene, '_AAseqs_mafft.fasta ',
                     '~/hbv_covar3/QC/step4_translate_gene/HBV_gene', gene, '_AAseqs_mafft_noShift.fasta'))
        system(paste0('cp ', '~/hbv_covar3/QC/step4_translate_gene/HBV_gene', gene, '_AAseqs.fasta ',
                     '~/hbv_covar3/QC/step4_translate_gene/HBV_gene', gene, '_AAseqs_noShift.fasta'))
        system(paste0('cp ', '~/hbv_covar3/QC/step4_translate_gene/HBV_gene', gene, '_seqs.fasta ',
                     '~/hbv_covar3/QC/step4_translate_gene/HBV_gene', gene, '_seqs_noShift.fasta'))
    }
}